In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [14]:
f = 'weatherHistory.csv'

weather = pd.read_csv(f)
weather.head(10)

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
5,2006-04-01 05:00:00.000 +0200,Partly Cloudy,rain,9.222222,7.111111,0.85,13.9587,258.0,14.9569,0.0,1016.66,Partly cloudy throughout the day.
6,2006-04-01 06:00:00.000 +0200,Partly Cloudy,rain,7.733333,5.522222,0.95,12.3648,259.0,9.9820,0.0,1016.72,Partly cloudy throughout the day.
7,2006-04-01 07:00:00.000 +0200,Partly Cloudy,rain,8.772222,6.527778,0.89,14.1519,260.0,9.9820,0.0,1016.84,Partly cloudy throughout the day.
8,2006-04-01 08:00:00.000 +0200,Partly Cloudy,rain,10.822222,10.822222,0.82,11.3183,259.0,9.9820,0.0,1017.37,Partly cloudy throughout the day.
9,2006-04-01 09:00:00.000 +0200,Partly Cloudy,rain,13.772222,13.772222,0.72,12.5258,279.0,9.9820,0.0,1017.22,Partly cloudy throughout the day.


In [15]:
weather['Formatted Date'].min()

'2006-01-01 00:00:00.000 +0100'

In [16]:
weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'])

# Set 'Formatted Date' as index
weather.set_index('Formatted Date', inplace=True)

weather = weather.sort_index()

In [17]:
missing_values = weather.isnull().sum()
print(missing_values)
weather['Precip Type'].unique()



Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64


array(['rain', 'snow', nan], dtype=object)

In [18]:
# Drop columns
weather.drop(columns=['Apparent Temperature (C)', 'Precip Type', 'Loud Cover', 'Daily Summary'], inplace=True)

# Verify changes
print(weather.head())

                                 Summary  Temperature (C)  Humidity  \
Formatted Date                                                        
2006-01-01 00:00:00+01:00  Partly Cloudy         0.577778      0.89   
2006-01-01 01:00:00+01:00  Mostly Cloudy         1.161111      0.85   
2006-01-01 02:00:00+01:00  Mostly Cloudy         1.666667      0.82   
2006-01-01 03:00:00+01:00       Overcast         1.711111      0.82   
2006-01-01 04:00:00+01:00  Mostly Cloudy         1.183333      0.86   

                           Wind Speed (km/h)  Wind Bearing (degrees)  \
Formatted Date                                                         
2006-01-01 00:00:00+01:00            17.1143                   140.0   
2006-01-01 01:00:00+01:00            16.6152                   139.0   
2006-01-01 02:00:00+01:00            20.2538                   140.0   
2006-01-01 03:00:00+01:00            14.4900                   140.0   
2006-01-01 04:00:00+01:00            13.9426                   134.0  

In [19]:
wind_degrees = weather['Wind Bearing (degrees)']

# Define function to map wind bearings to categories
def map_wind_direction(degrees):
    directions = ['N', 'NNE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    degree_ranges = [(348.75, 11.25), (11.25, 33.75), (33.75, 56.25), (56.25, 78.75), (101.25, 123.75),
                     (123.75, 146.25), (146.25, 168.75), (168.75, 191.25), (191.25, 213.75), (213.75, 236.25),
                     (236.25, 258.75), (258.75, 281.25), (281.25, 303.75), (303.75, 326.25), (326.25, 348.75)]
    for direction, (start, end) in zip(directions, degree_ranges):
        if start <= degrees < end:
            return direction
    return directions[0]  # Default to 'N' if not in any range

# Map wind direction
weather['Wind Direction'] = weather['Wind Bearing (degrees)'].apply(map_wind_direction)

# Perform one-hot encoding
weather = pd.get_dummies(weather, columns=['Wind Direction'], prefix='Wind')

# Drop original 'Wind Bearing (degrees)' column
weather.drop(columns=['Wind Bearing (degrees)'], inplace=True)

weather['Summary'].unique()

weather_summary = weather['Summary']

# Perform one-hot encoding for 'Summary' column
weather = pd.get_dummies(weather, columns=['Summary'], prefix='Summary')

# Verify changes
print(weather.head())

                           Temperature (C)  Humidity  Wind Speed (km/h)  \
Formatted Date                                                            
2006-01-01 00:00:00+01:00         0.577778      0.89            17.1143   
2006-01-01 01:00:00+01:00         1.161111      0.85            16.6152   
2006-01-01 02:00:00+01:00         1.666667      0.82            20.2538   
2006-01-01 03:00:00+01:00         1.711111      0.82            14.4900   
2006-01-01 04:00:00+01:00         1.183333      0.86            13.9426   

                           Visibility (km)  Pressure (millibars)  Wind_E  \
Formatted Date                                                             
2006-01-01 00:00:00+01:00           9.9820               1016.66       0   
2006-01-01 01:00:00+01:00           9.9015               1016.15       0   
2006-01-01 02:00:00+01:00           9.9015               1015.87       0   
2006-01-01 03:00:00+01:00           9.9015               1015.56       0   
2006-01-01 04:00:0

### Model

In [20]:
# Define window size (number of time steps to consider)
window_size = 24 * 3  # 3 days (24 hours each)

# Function to create input-output pairs from time series data
def create_dataset(X, y, window_size):
    X_data, y_data = [], []
    for i in range(len(X) - window_size):
        X_data.append(X[i:i+window_size])
        y_data.append(y[i+window_size])
    return np.array(X_data), np.array(y_data)

# Prepare input-output pairs
X, y = create_dataset(weather.values, weather['Temperature (C)'].values, window_size)


In [21]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Exclude dummy variables from scaling
continuous_columns = ['Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Visibility (km)', 'Pressure (millibars)']
X_train_continuous = X_train[:, :, :len(continuous_columns)]
X_test_continuous = X_test[:, :, :len(continuous_columns)]

# Scale continuous features
scaler = StandardScaler()
X_train_continuous_scaled = scaler.fit_transform(X_train_continuous.reshape(-1, X_train_continuous.shape[-1])).reshape(X_train_continuous.shape)
X_test_continuous_scaled = scaler.transform(X_test_continuous.reshape(-1, X_test_continuous.shape[-1])).reshape(X_test_continuous.shape)

# Combine scaled continuous features with dummy variables
X_train_scaled = np.concatenate((X_train_continuous_scaled, X_train[:, :, len(continuous_columns):]), axis=-1)
X_test_scaled = np.concatenate((X_test_continuous_scaled, X_test[:, :, len(continuous_columns):]), axis=-1)

In [25]:
# Reshape data for LSTM input (samples, time steps, features)
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], window_size, -1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], window_size, -1)


In [26]:
X_train_scaled

array([[[-1.18409869,  0.81359287,  0.89894815, ...,  0.        ,
          0.        ,  0.        ],
        [-1.12374756,  0.61050549,  0.8274579 , ...,  0.        ,
          0.        ,  0.        ],
        [-1.07144324,  0.45818995,  1.34864483, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-1.01166688,  0.96590841, -0.86985879, ...,  0.        ,
          0.        ,  0.        ],
        [-1.01051733,  0.96590841, -1.08894179, ...,  0.        ,
          0.        ,  0.        ],
        [-1.01626506,  0.81359287, -0.64616352, ...,  0.        ,
          0.        ,  0.        ]],

       [[-1.12374756,  0.61050549,  0.8274579 , ...,  0.        ,
          0.        ,  0.        ],
        [-1.07144324,  0.45818995,  1.34864483, ...,  0.        ,
          0.        ,  0.        ],
        [-1.06684506,  0.45818995,  0.52304784, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-1.01051733,  0.96590841, -1.08894179, ...,  

In [27]:

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))  # Output layer with 1 neuron for temperature prediction
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test_scaled)

# Print some predictions
print("Some predictions:")
for i in range(10):
    print(f"Predicted: {predictions[i][0]}, Actual: {y_test[i]}")



Epoch 1/50

2410/2410 [==============================] - 61s 24ms/step - loss: 19.3647
Epoch 2/50
2410/2410 [==============================] - 57s 23ms/step - loss: 1.9790
Epoch 3/50
2410/2410 [==============================] - 46s 19ms/step - loss: 1.2910
Epoch 4/50
2410/2410 [==============================] - 46s 19ms/step - loss: 1.1362
Epoch 5/50
2410/2410 [==============================] - 49s 20ms/step - loss: 1.0644
Epoch 6/50
2410/2410 [==============================] - 52s 21ms/step - loss: 1.0252
Epoch 7/50
2410/2410 [==============================] - 52s 22ms/step - loss: 0.9984
Epoch 8/50
2410/2410 [==============================] - 60s 25ms/step - loss: 0.9798
Epoch 9/50
2410/2410 [==============================] - 50s 21ms/step - loss: 0.9571
Epoch 10/50
2410/2410 [==============================] - 50s 21ms/step - loss: 0.9441
Epoch 11/50
2410/2410 [==============================] - 50s 21ms/step - loss: 0.9301
Epoch 12/50
2410/2410 [==============================] - 51